# Use MLflow in MONAI Workflow

[MLflow](https://mlflow.org/) is an experiment management tool that can be used for logging experiment details and results in machine learning experiments. The MONAI workflow integrates mlflow as a part of it to make it convenient for users recording their experiments. This tutorial shows how to enable it in MONAI bundle workflow from three aspects.
1. Use MLflow in MONAI bundle by default.
2. Use MLflow in MONAI bundle with a config file.
3. Use MLflow in parsed MONAI bundle with python code.

This tutorial takes the [3D spleen segmentation task](https://github.com/Project-MONAI/tutorials/blob/main/3d_segmentation/spleen_segmentation_3d.ipynb) as an example. In order to quickly verify the MLflow function, each example will only run 5 epochs.

## Setup Environment

`MLFlow` comes as part of the `monai[all]` installation. For official documentation on MLFlow's experiment management functionalities, click [here](https://www.mlflow.org/docs/latest/tracking.html).

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"

## Setup imports

In [2]:
import os
import monai
import tempfile
from monai.apps import download_and_extract
from monai.bundle import ConfigParser
from monai.handlers import MLFlowHandler

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable. This allows you to save results and reuse downloads. If not specified a temporary directory will be used.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

env: MONAI_DATA_DIRECTORY=/workspace/data/medical
/workspace/data/medical


## Download spleen dataset
Downloads and extracts the dataset. The dataset comes from http://medicaldecathlon.com/.

In [4]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar"
md5 = "410d4a301da4e5b2f6f86ec3ddba524e"

compressed_file = os.path.join(root_dir, "Task09_Spleen.tar")
data_dir = os.path.join(root_dir, "Task09_Spleen")
print(data_dir)
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

/workspace/data/medical/Task09_Spleen


## Use MLflow in MONAI bundle

In this part, we will take the [spleen segmentation bundle](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation) as an example to show how to enable MLflow in it. Typically there are two ways to enable MLflow in a bundle training process. The easiest way is to add `--tracking "mlflow"` at the end of the command line. Some extra parameters like `tracking_uri` and `experiment_name` can also be added this way. The second is to add a config json file as input. In this file, users can define their own setting on MLflow. 

### Download spleen segmentation bundle

In [5]:
monai.bundle.download(name="spleen_ct_segmentation", version="0.3.7", bundle_dir="./")

2022-12-09 03:08:01,304 - INFO - --- input summary of monai.bundle.scripts.download ---
2022-12-09 03:08:01,305 - INFO - > name: 'spleen_ct_segmentation'
2022-12-09 03:08:01,306 - INFO - > version: '0.3.7'
2022-12-09 03:08:01,306 - INFO - > bundle_dir: './'
2022-12-09 03:08:01,306 - INFO - > source: 'github'
2022-12-09 03:08:01,307 - INFO - > repo: 'Project-MONAI/model-zoo/hosting_storage_v1'
2022-12-09 03:08:01,307 - INFO - > progress: True
2022-12-09 03:08:01,307 - INFO - ---


2022-12-09 03:08:01,308 - INFO - Expected md5 is None, skip md5 check for file spleen_ct_segmentation_v0.3.7.zip.
2022-12-09 03:08:01,308 - INFO - File exists: spleen_ct_segmentation_v0.3.7.zip, skipped downloading.
2022-12-09 03:08:01,308 - INFO - Writing into directory: ..


### Run spleen bundle with MLflow parameter

The command line in the next cell is the easiest way to run the spleen segementation training  bundle with mlflow. Please modify the `--dataset_dir` with your own path of dataset. The parameter `--tracking "mlflow"` at the end of the original command is to enable the mlflow during training. Parameters `--tracking_uri`, `--experiment_name` and `--run_name` can also be added and modified to change the tracking uri, experiment name and run name of mlflow. To enable MLflow in multi-gpu training is as the same with single gpu by adding `--tracking "mlflow"` to the end of command line. 

A `mlruns` folder will be created in the `spleen_ct_segmentation/eval` folder during the running. Running the command `mlflow ui` in this folder can set a webpage UI for tracking. By default, the address will be `http://127.0.0.1:5000`. If there is a confliction of port or host address, `--port` and `--host` parameters can be modified to new one. 
Here is the tracking result.
![image](./extra_pics/mlflow_default_result.png)

In [6]:
!cd spleen_ct_segmentation;python -m monai.bundle run training \
    --meta_file configs/metadata.json \
    --config_file configs/train.json \
    --logging_file configs/logging.conf \
    --bundle_root ./ \
    --dataset_dir /workspace/data/medical/Task09_Spleen \
    --train#trainer#max_epochs 10 \
    --tracking "mlflow"

2022-12-09 03:08:04,152 - INFO - --- input summary of monai.bundle.scripts.run ---
2022-12-09 03:08:04,152 - INFO - > runner_id: 'training'
2022-12-09 03:08:04,152 - INFO - > meta_file: 'configs/metadata.json'
2022-12-09 03:08:04,152 - INFO - > config_file: 'configs/train.json'
2022-12-09 03:08:04,152 - INFO - > logging_file: 'configs/logging.conf'
2022-12-09 03:08:04,152 - INFO - > tracking: 'mlflow'
2022-12-09 03:08:04,152 - INFO - > bundle_root: './'
2022-12-09 03:08:04,152 - INFO - > dataset_dir: '/workspace/data/medical/Task09_Spleen'
2022-12-09 03:08:04,152 - INFO - > train#trainer#max_epochs: 10
2022-12-09 03:08:04,152 - INFO - ---


2022-12-09 03:08:04,152 - INFO - set logging properties based on config: configs/logging.conf.
Loading dataset: 100%|████████████████████████████| 9/9 [00:10<00:00,  1.13s/it]
2022-12-09 03:08:37,943 - ignite.engine.engine.SupervisedTrainer - INFO - Engine run resuming from iteration 0, epoch 0 until 10 epochs
2022/12/09 03:08:37 INFO mlflow.trackin

### Run spleen bundle with a MLflow config file

The other way to run MLflow with bundle is to pass a JSON config file to `--tracking` parameter. In this file, a `mlflow_hander`, which is a handler to leverage MLflow in MONAI bundle, should be defined to enable tracking. More parameters and details can be added through this way. There is an example JSON named `mlflow_example.json` in this folder for reference. When writting the config JSON in multi-gpu environment, please note to use `_disabled_` parameter as shown in the example JSON to only use MLflow in the first gpu.

The next cell contains a command line to run spleen segmentation training with the given config JSON. As the experiment finished, it will be logged as shown below. And it is different from the defualt one by `run_name` and `parameters`, since we changed these in the config file.

![image](./extra_pics/mlflow_config_result.png)

In [7]:
!cd spleen_ct_segmentation;python -m monai.bundle run training \
    --meta_file configs/metadata.json \
    --config_file configs/train.json \
    --logging_file configs/logging.conf \
    --bundle_root ./ \
    --dataset_dir /workspace/data/medical/Task09_Spleen \
    --train#trainer#max_epochs 10 \
    --tracking ../mlflow_example.json

2022-12-09 03:09:18,211 - INFO - --- input summary of monai.bundle.scripts.run ---
2022-12-09 03:09:18,211 - INFO - > runner_id: 'training'
2022-12-09 03:09:18,211 - INFO - > meta_file: 'configs/metadata.json'
2022-12-09 03:09:18,211 - INFO - > config_file: 'configs/train.json'
2022-12-09 03:09:18,211 - INFO - > logging_file: 'configs/logging.conf'
2022-12-09 03:09:18,211 - INFO - > tracking: '../mlflow_example.json'
2022-12-09 03:09:18,211 - INFO - > bundle_root: './'
2022-12-09 03:09:18,211 - INFO - > dataset_dir: '/workspace/data/medical/Task09_Spleen'
2022-12-09 03:09:18,211 - INFO - > train#trainer#max_epochs: 10
2022-12-09 03:09:18,211 - INFO - ---


2022-12-09 03:09:18,211 - INFO - set logging properties based on config: configs/logging.conf.
Loading dataset: 100%|████████████████████████████| 9/9 [00:10<00:00,  1.11s/it]
2022-12-09 03:09:51,801 - ignite.engine.engine.SupervisedTrainer - INFO - Engine run resuming from iteration 0, epoch 0 until 10 epochs
2022-12-09 03:09:55,066

### Run parsed spleen segmentation bundle with mlflow_handler

In this part, we use the parsed trainer from spleen bundle to show how to add mlflow_handler to a monai engine in python code. However, users can also write their own workflow in python code from beginning and reference to this part to add mlflow_handler. 
The recorded results are shown below:
![image](./extra_pics/mlflow_python_result.png)

In [8]:
tracking_uri = "./spleen_ct_segmentation/eval/mlruns"
ml_tracking = MLFlowHandler(
    tracking_uri=tracking_uri,
    experiment_name="ParsedExperiment",
    run_name="Parsed1",
    tag_name="train_loss",
    iteration_log=True,
    epoch_log=True,
    output_transform=monai.handlers.from_engine(["loss"], first=True),
)
parser = ConfigParser()
parser.read_config(f="./spleen_ct_segmentation/configs/train.json")
parser.read_meta(f="./spleen_ct_segmentation/configs/metadata.json")
parser.update({"train#trainer#max_epochs": 10, "dataset_dir": data_dir})

trainer = parser.get_parsed_content("train#trainer")
ml_tracking.attach(trainer)
trainer.run()

Loading dataset: 100%|██████████| 9/9 [00:10<00:00,  1.12s/it]

2022-12-09 03:11:03,853 - Engine run resuming from iteration 0, epoch 0 until 10 epochs



2022/12/09 03:11:03 INFO mlflow.tracking.fluent: Experiment with name 'ParsedExperiment' does not exist. Creating a new experiment.


2022-12-09 03:11:06,452 - Epoch: 1/10, Iter: 1/16 -- train_loss: 1.2814 
2022-12-09 03:11:06,553 - Epoch: 1/10, Iter: 2/16 -- train_loss: 1.2986 
2022-12-09 03:11:06,634 - Epoch: 1/10, Iter: 3/16 -- train_loss: 1.2531 
2022-12-09 03:11:06,739 - Epoch: 1/10, Iter: 4/16 -- train_loss: 1.2530 
2022-12-09 03:11:06,843 - Epoch: 1/10, Iter: 5/16 -- train_loss: 1.2678 
2022-12-09 03:11:07,036 - Epoch: 1/10, Iter: 6/16 -- train_loss: 1.2561 
2022-12-09 03:11:07,121 - Epoch: 1/10, Iter: 7/16 -- train_loss: 1.2565 
2022-12-09 03:11:07,203 - Epoch: 1/10, Iter: 8/16 -- train_loss: 1.2506 
2022-12-09 03:11:07,304 - Epoch: 1/10, Iter: 9/16 -- train_loss: 1.2100 
2022-12-09 03:11:07,401 - Epoch: 1/10, Iter: 10/16 -- train_loss: 1.2038 
2022-12-09 03:11:07,496 - Epoch: 1/10, Iter: 11/16 -- train_loss: 1.1974 
2022-12-09 03:11:07,591 - Epoch: 1/10, Iter: 12/16 -- train_loss: 1.1782 
2022-12-09 03:11:07,686 - Epoch: 1/10, Iter: 13/16 -- train_loss: 1.1557 
2022-12-09 03:11:07,781 - Epoch: 1/10, Iter: 14